# Agentes React

## Setup

In [1]:
from dotenv import find_dotenv, load_dotenv
from langchain import hub
from langchain.agents import (
    AgentExecutor,
    create_react_agent,
    initialize_agent,
    Tool,
)
from langchain.prompts import PromptTemplate
from langchain.tools import WikipediaQueryRun, DuckDuckGoSearchRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_openai import OpenAI

In [2]:
_ = load_dotenv(find_dotenv('.env'))

In [3]:
openai = OpenAI(model='gpt-3.5-turbo-instruct', temperature=0)

## Assistente financeiro

In [15]:
PROMPT = """Como assistente financeiro pessoal, ajude a responder a pergunta e dar dicas.
Perguntas: {q}"""

prompt_template = PromptTemplate.from_template(PROMPT)

In [16]:
react_instructions = hub.pull('hwchase17/react')
print(react_instructions)

/home/rwallan/git/langchain-course/.venv/lib/python3.13/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'} template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'


In [17]:
# Ferramenta 1
python_repl_tool = Tool(
    name='Python REPL',
    func=PythonREPLTool().run,
    description="""Qualquer tipo de cálculo deve usar essa ferramenta.
Você não deve realizar o cálculo diretamente, insira o código Python."""
)

In [18]:
# Ferramenta 2
wikipedia_tool = Tool(
    name='Busca DuckDuckGo',
    func=DuckDuckGoSearchRun().run,
    description="""Útil para coletar informações e dicas de economia e opções de investimento.
Você deve sempre pesquisar na internet as melhores dicas usando esta ferramenta, não responda diretamente.
Sua resposta deve informar que há informações coletadas pela internet"""
)

In [ ]:
tools = [python_repl_tool, wikipedia_tool]
agent = create_react_agent(openai, tools, react_instructions)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=10
)

# print(agent_executor)

In [22]:
question = """Eu ganho R$ 6000 por mês mas o total das minhas despesas é de R$ 2500 mais 1000 de aluguel.
Quais dicas de investimento você me dá?"""

In [23]:
output = agent_executor.invoke({'input': prompt_template.format(q=question)})
print(output)



> Entering new AgentExecutor chain...
 Eu devo usar a ferramenta Python REPL para realizar cálculos e a ferramenta Busca DuckDuckGo para coletar informações e dicas de investimento.
Action: Python REPL
Action Input: 6000 - (2500 + 1000)2500 é o total das minhas despesas e 1000 é o valor do aluguel, então o resultado é o valor que sobra para investir.
Action: Busca DuckDuckGo
Action Input: "dicas de investimento"

/home/rwallan/.pyenv/versions/3.13.2/lib/python3.13/asyncio/events.py:89: UserWarning: lxml is not installed. Using backend='api'.
  self._context.run(self._callback, *self._args)


Para que você fique dentro de uma zona de conforto de segurança e rentabilidade, a melhor alternativa são os investimentos de renda fixa. Neste caso, você compra a dívida de um banco (como é o caso do CDB ) ou até mesmo do governo (como no caso do Tesouro Direto ) e depois é pago ao final de um prazo com as correções dos juros do ... A Forbes separou 12 dicas de investimento sobre a importância da diversificação, compreensão de risco, gerenciamento de taxas e manutenção de uma perspectiva de longo prazo. Assim, você pode fazer seus investimentos com confiança, preparando-se para um crescimento sustentável e sucesso financeiro ao longo do tempo. Dessa forma, você não depende de um único investimento para obter bons resultados. Outra dica importante é acompanhar o mercado regularmente. Mesmo que você opte por investimentos de longo prazo, estar informado sobre as tendências e mudanças econômicas pode ajudar a tomar decisões mais assertivas. "Investir é como plantar uma árvore. Uma das me